# EE-627: Project by Ex-Machina Group

## Imports + Set-Up Variables

#### STEP 1: Use the training history of users to find the related rating scores related to the track IDs with the testing users. 

In [10]:
import numpy as np
import csv
PRINT_BOOLEAN = False;

def smart_print(statement, arg1=""):
    if (PRINT_BOOLEAN == True):
        print(statement,arg1)

# dataDir='/YOUR_DIRECTORY/Project/data_in_matrixForm/'

# Single User Testing Data
if (PRINT_BOOLEAN == True):
    test_file_source   = 'single_user_test.txt'
    train_file_source  = 'single_user_train.txt'
# Official Testing Data
else:
    test_file_source   = '/content/drive/MyDrive/SIT_MSAAI/fall2021/627 Big Data/Assignment 5/testTrack_hierarchy.txt'
    train_file_source  = '/content/drive/MyDrive/SIT_MSAAI/fall2021/627 Big Data/Assignment 5/trainIdx2_matrix.txt'
    
output_file_source = 'out_CombineTrainTest.txt'

read_test_file    = open(test_file_source,  'r')
read_train_file   = open(train_file_source, 'r')
train_data_line   = read_train_file.readline()
write_output_file = open(output_file_source, 'w')

# Initialize Variables, for set-up
trackID_vec  = [0]*6
albumID_vec  = [0]*6
artistID_vec = [0]*6
rows, cols = (6, 30) # 30 columns for genres (ample amount of space) and 6 rows for a total of 6 tracks
genreID_vec = [[0]*cols]*rows
lastUserID   = -1
user_rating_inTrain = np.zeros(shape=(6,3))

print("Starting...Please Wait...")
# For each line in the test data file, start reading
for line in read_test_file:
    temp_genreID_vec = [0]*cols
    
    # Break down the data from a single line
    arr_test = line.strip().split('|') # split the data, based on the delimiter "|"
    smart_print("Split Input Test Array:",arr_test)
    smart_print("Length of Input Test Array:",len(arr_test))
    for test_id in range (len(arr_test)) :
        userID   = arr_test[0] # First data is always User ID
        trackID  = arr_test[1] # Second data is always Track ID
        albumID  = arr_test[2] # Third data is always Album ID
        artistID = arr_test[3] # Fourth data is always Artist ID
        if (test_id >= 4):
            temp_genreID_vec[(test_id-4)] = arr_test[test_id] # Populate the temporary genre ID List for a single track
    
    smart_print("Individual Genre Vector:", temp_genreID_vec)
    smart_print("\n")
    # Check to see if current UserID is same as previous User ID
    # IF not the same, reset the counters, and re-initialize the user_rating matrix
    if (userID != lastUserID): 
        ii = 0
        user_rating_inTrain = np.zeros(shape=(6,3))
    
    # populate track, album, and artist vectors with the current user's preference
    trackID_vec[ii]  = trackID
    albumID_vec[ii]  = albumID
    artistID_vec[ii] = artistID
    genreID_vec[ii]  = temp_genreID_vec
    # increment the current user's counter +1
    ii = ii + 1
    lastUserID = userID

    # After 6 tracks have been read for a single User, start looking at the training data
    if (ii == 6): 
        smart_print("Individual Genre Vector:", genreID_vec)
        smart_print("\n")
        while (train_data_line): # for each line, until the end, in the training data file
        # for train_data_line in read_train_file:
            # Break down the data from a single line
            arr_train = train_data_line.strip().split('|') # split the data, based on the delimiter "|"
            trainUserID = arr_train[0] # First data is always User ID
            trainItemID = arr_train[1] # Second data is always Item ID
            trainRating = arr_train[2] # Third data is always Rating
            
            # Go to the next line to read
            train_data_line = read_train_file.readline()

            # if training user is less than current testing user, we ignore and continue
            if trainUserID < userID:
                continue
            
            # if training user is same as testing user
            if trainUserID == userID:
                for nn in range(0, 6): # loop through all 6 tracks
                    
                    # check if the training's ITEM_ID exists in testing's ALBUM_ID
                    if trainItemID == albumID_vec[nn]: 
                        user_rating_inTrain[nn, 0] = trainRating
                        
                    # check if the training's ITEM_ID exists in testing's ARTIST_ID
                    if trainItemID==artistID_vec[nn]: 
                        user_rating_inTrain[nn, 1] = trainRating
                        
                    # check if the training's ITEM_ID exists in testing's GENRE_IDs
                    for gen_id in range (len(genreID_vec[nn])):
                        if trainItemID == genreID_vec[nn][gen_id]: 
                            user_rating_inTrain[nn, 2] = str(int(user_rating_inTrain[nn, 2]) + int(trainRating))
                            smart_print((nn,trainItemID), trainRating)
            
            # if it went to the next user, it's time to print out the results
            if trainUserID > userID:
                smart_print(user_rating_inTrain)
                for nn in range(0, 6):
                    outStr=str(userID) + '|' + str(trackID_vec[nn])+ '|' + str(user_rating_inTrain[nn,0]) + '|' + str(user_rating_inTrain[nn, 1]) + '|' + str(user_rating_inTrain[nn, 2])
                    smart_print("OutputString:",outStr)
                    write_output_file.write(outStr + '\n')
                break

# Close out the opened files
read_test_file.close()
read_train_file.close()
write_output_file.close()
print("Done!")

Starting...Please Wait...
Done!


#### STEP 2: Define your rule to find 3 tracks with "1" and 3 with "0". 

In [ ]:
import numpy as np
import csv

PRINT_BOOLEAN = False;

# Define Inputs and Read/Write Files
in_file = 'out_CombineTrainTest.txt' #Using the output file we obtained in Step 1
out_file = 'output2.txt'             #New Output file, for Step 2
combinedTrainTest_file = open(in_file, 'r')
fOut = open(out_file, 'w')

f = open('answer.csv', 'w', newline='')
writer = csv.writer(f)
writer.writerow(['TrackID', 'Predictor'])

# Initialize Variables
lastUserID = -1
#firstRun   = -1
trackID_vec = [0]*6
score_result_vec  = [0]*6

# Weight Multipliers
ARTIST_WGT = 0.25 # 
GENRE_WGT = 0.05 # 5% gave best results thus far

print("Starting...Please Wait...")
# Traverse through each line in combinedTrainTest File - that we obtained in Step 1
for line in combinedTrainTest_file:
    # Break down the data from a single line
    arr_ans = line.strip().split('|')
    #smart_print("Split Input Test Array:",arr_ans)
    userID         = arr_ans[0]  # First data is always User ID
    trackID        = arr_ans[1]  # First data is always Track ID
    albumID_score  = arr_ans[2]  # First data is always User ID
    artistID_score = arr_ans[3]  # First data is always User ID
    genre_score    = arr_ans[4]  # First data is always User ID
    
    # Check to see if current UserID is same as previous User ID
    # IF not the same, reset the counters, and re-initialize the user_rating matrix
    if (userID != lastUserID):
        ii = 0 
        user_rating = np.zeros(shape = (6,3))
    
    new_num = str(userID) + "_"
    trackID_vec[ii] = trackID
    score_result_vec[ii] = float(albumID_score) + ARTIST_WGT*float(artistID_score) + GENRE_WGT*float(genre_score)
    # increment the current user's counter +1
    ii = ii + 1
    lastUserID = userID

    # Traverse through each of the 6 songs, and analyze them
    if (ii == 6): 
        for nn in range(0, 6):
            user_rating[nn][0] = userID
            user_rating[nn][1] = trackID_vec[nn]
            user_rating[nn][2] = score_result_vec[nn]
            
        # Numpy's Sorting Method - Ascending
        # LINK: https://opensourceoptions.com/blog/sort-numpy-arrays-by-columns-or-rows/
        user_rating = user_rating[user_rating[:, 2].argsort()]
        for nn in range(6):
            if (nn < 3):
                user_rating[nn][2] = 0
            else:
                user_rating[nn][2] = 1
        
            out = userID + '|' + str(int(user_rating[nn, 1])) + '|' + str(int(score_result_vec[nn])) + '|' + str(int(user_rating[nn, 2]))
            fOut.write(out + '\n')
            writer.writerow([new_num + str(int(user_rating[nn,1])), int(user_rating[nn, 2])])
            
print("Done!")
combinedTrainTest_file.close()
f.close()

Starting...Please Wait...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
